In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [2]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [3]:
len(raw_test)

367

In [4]:
--version

NameError: name 'version' is not defined

In [7]:
-version

NameError: name 'version' is not defined

In [8]:
python__version

NameError: name 'python__version' is not defined

In [11]:
python __version

SyntaxError: invalid syntax (<ipython-input-11-d35a016e4840>, line 1)

In [14]:
import python

ModuleNotFoundError: No module named 'python'